In [1]:
import os
import torch
from configparser import ConfigParser, ExtendedInterpolation
from base64 import b64encode
from cotracker.utils.visualizer import Visualizer, read_video_from_path, read_images_from_path
from IPython.display import HTML
from cotracker.predictor import CoTrackerPredictor

config = ConfigParser(interpolation=ExtendedInterpolation())
config.read("config.ini")

['config.ini']

In [2]:
Input_ImgDir = config.get('Dir', 'INPUT_DIR')
print(Input_ImgDir)

/home/maneesh/Desktop/LAB2.0/DATA-FDCL/Real_Data/Test_2/


In [3]:
#video = read_video_from_path('./assets/apple.mp4')
video = read_images_from_path(Input_ImgDir, 1, 150)
video = torch.from_numpy(video).permute(0, 3, 1, 2)[None].float()

In [4]:
def show_video(video_path):
    video_file = open(video_path, "r+b").read()
    video_url = f"data:video/mp4;base64,{b64encode(video_file).decode()}"
    return HTML(f"""<video width="640" height="480" autoplay loop controls><source src="{video_url}"></video>""")
 

Import CoTrackerPredictor and create an instance of it. We'll use this object to estimate tracks:

In [5]:
model = CoTrackerPredictor(checkpoint='cotracker/ckpt/cotracker_stride_4_wind_8.pth')

In [6]:
if torch.cuda.is_available():
    model = model.cuda()
    video = video.cuda()

In [7]:
queries = torch.tensor([
    [0., 452., 111.],  # point tracked from the first frame
    [0., 531., 192.], # frame number 10
    [0., 452., 191.], # ...
    [0., 533., 112.],  # point tracked from the first frame

])
if torch.cuda.is_available():
    queries = queries.cuda()
    #model = model.cuda()
    # video = video.cuda()

In [8]:
# queries = torch.load('q.pt')
# if torch.cuda.is_available():
#     queries = queries.cuda()
# print(queries)

In [9]:
pred_tracks, pred_visibility = model(video, queries=queries[None])

print(pred_tracks)

tensor([[[[452.0172, 111.0173],
          [531.0002, 191.9876],
          [451.9821, 190.9396],
          [532.9894, 112.0374]],

         [[452.0750, 110.9472],
          [531.0058, 191.9603],
          [452.0400, 190.9190],
          [532.9982, 111.9660]],

         [[452.0651, 110.9705],
          [531.0154, 191.9975],
          [452.0574, 190.9826],
          [533.0140, 112.0247]],

         ...,

         [[316.0435, 262.1841],
          [323.4482, 275.5367],
          [313.7953, 274.3712],
          [327.5273, 264.0331]],

         [[309.7959, 269.8309],
          [316.2903, 282.8835],
          [306.8087, 281.5778],
          [321.0040, 271.8475]],

         [[302.1252, 277.6769],
          [309.1065, 290.6748],
          [299.1593, 289.1313],
          [313.8086, 279.7300]]]], device='cuda:0')


In [10]:
vis = Visualizer(
    save_dir='./videos',
    linewidth=1,
    mode='cool',
    tracks_leave_trace=-1
)
vis.visualize(
    video=video,
    tracks=pred_tracks,
    visibility=pred_visibility,
    filename='queries');

Video saved to ./videos/queries_pred_track.mp4


In [11]:
show_video("./videos/queries_pred_track.mp4")